In [1]:
import duckdb
import glob

file_path = "C:/Users/Ida/.cache/kagglehub/datasets/jeffsinsel/nyc-fhvhv-data/versions/4"

#laptop
# file_path = "C:/Users/idavo/.cache/kagglehub/datasets/jeffsinsel/nyc-fhvhv-data/versions/4"

In [2]:
selected_columns = ["pickup_datetime", "dropoff_datetime","trip_miles", "trip_time", "base_passenger_fare", "congestion_surcharge", "hvfhs_license_num", "tolls", "sales_tax", "tips", "shared_match_flag"]
parquet_files = glob.glob(f"{file_path}/*.parquet")
output_file = "nyc_data.parquet"

con = duckdb.connect()
con.execute(f"CREATE TABLE merged AS SELECT {', '.join(selected_columns)} FROM parquet_scan('{file_path}/*.parquet') WHERE 1=0")

for file in parquet_files:
    con.execute(f"""
        INSERT INTO merged
        SELECT {', '.join(selected_columns)}
        FROM parquet_scan('{file}')
        WHERE pickup_datetime < dropoff_datetime AND congestion_surcharge IS NOT NULL
    """)

con.execute(f"COPY merged TO '{output_file}' (FORMAT 'parquet')")
con.close()

In [3]:
test = duckdb.sql(f"""
    SELECT * FROM 'nyc_data.parquet'
    LIMIT 20
""")

print(test)

┌─────────────────────┬─────────────────────┬────────────┬───────────┬─────────────────────┬──────────────────────┬───────────────────┬────────┬───────────┬────────┬───────────────────┐
│   pickup_datetime   │  dropoff_datetime   │ trip_miles │ trip_time │ base_passenger_fare │ congestion_surcharge │ hvfhs_license_num │ tolls  │ sales_tax │  tips  │ shared_match_flag │
│      timestamp      │      timestamp      │   double   │   int64   │       double        │        double        │      varchar      │ double │  double   │ double │      varchar      │
├─────────────────────┼─────────────────────┼────────────┼───────────┼─────────────────────┼──────────────────────┼───────────────────┼────────┼───────────┼────────┼───────────────────┤
│ 2019-02-01 00:05:18 │ 2019-02-01 00:14:57 │       2.45 │       579 │                9.35 │                  0.0 │ HV0003            │    0.0 │      0.83 │    0.0 │ N                 │
│ 2019-02-01 00:41:29 │ 2019-02-01 00:49:39 │       1.71 │       490 │